In [1]:
import numpy as np
import pandas as pd
from scipy.sparse.linalg import svds

In [2]:
book_df = pd.read_csv('Books.csv')
ratings_df = pd.read_csv('Ratings.csv').sample(60000)
user_df = pd.read_csv('Users.csv')

<ipython-input-2-1407295537a8>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  book_df = pd.read_csv('Books.csv')


In [3]:
book_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271358 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


In [4]:
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 60000 entries, 388733 to 71170
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   User-ID      60000 non-null  int64 
 1   ISBN         60000 non-null  object
 2   Book-Rating  60000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.8+ MB


In [5]:
user_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   278858 non-null  int64  
 1   Location  278858 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


In [6]:
ratings_df.isnull().sum()


,0
User-ID,0
ISBN,0
Book-Rating,0


In [7]:
ratings_df = ratings_df[ratings_df['Book-Rating'] != 0]

In [8]:
book_df.isnull().sum()

,0
ISBN,0
Book-Title,0
Book-Author,2
Year-Of-Publication,0
Publisher,2
Image-URL-S,0
Image-URL-M,0
Image-URL-L,3


In [9]:
book_df.dropna(how='any', inplace=True)

In [10]:
user_df.isnull().sum()

,0
User-ID,0
Location,0
Age,110762


In [11]:
user_rating_df = ratings_df.merge(user_df, left_on = 'User-ID', right_on = 'User-ID')

In [12]:
ratings_df.head(10)

,User-ID,ISBN,Book-Rating
388733,93701,0399149155,9
361053,87141,006109174X,8
468283,112001,0373291205,9
30859,7181,8432038598,10
310997,74687,0385306040,8
624633,151503,0891076182,8
261033,60244,0698113608,9
769372,185677,0425158640,9
248010,56654,0446611085,5
157051,35857,0842350357,10


In [13]:
user_rating_df.head(10)

,User-ID,ISBN,Book-Rating,Location,Age
0,93701,0399149155,9,"murphyusboro, illinois, usa",NaN
1,87141,006109174X,8,"huntington beach, california, usa",NaN
2,112001,0373291205,9,"barren springs, virginia, usa",43.0
3,7181,8432038598,10,"oviedo, asturias, spain",24.0
4,74687,0385306040,8,"charlotte, north carolina, usa",54.0
5,151503,0891076182,8,"fairbanks, alaska, usa",33.0
6,60244,0698113608,9,"alvin, texas, usa",47.0
7,185677,0425158640,9,"tampa, florida, usa",NaN
8,56654,0446611085,5,"fort worth, texas, usa",NaN
9,35857,0842350357,10,"mentor, ohio, usa",36.0


In [14]:
book_user_rating = book_df.merge(user_rating_df, left_on = 'ISBN', right_on = 'ISBN')
book_user_rating = book_user_rating[['ISBN', 'Book-Title', 'Book-Author', 'User-ID', 'Book-Rating']]
book_user_rating.reset_index(drop=True, inplace=True)
book_user_rating.head(10)

,ISBN,Book-Title,Book-Author,User-ID,Book-Rating
0,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,110912,10
1,0440234743,The Testament,John Grisham,101209,7
2,0440234743,The Testament,John Grisham,133706,8
3,0440234743,The Testament,John Grisham,251122,9
4,0440234743,The Testament,John Grisham,54005,9
5,0440234743,The Testament,John Grisham,198046,8
6,0440234743,The Testament,John Grisham,108143,9
7,0440234743,The Testament,John Grisham,123430,5
8,0440234743,The Testament,John Grisham,44663,8
9,0440234743,The Testament,John Grisham,193529,7


In [15]:
book_user_rating.size

99955

In [16]:
d ={}
for i,j in enumerate(book_user_rating.ISBN.unique()):
    d[j] =i
book_user_rating['unique_id_book'] = book_user_rating['ISBN'].map(d)
book_user_rating.head(10)

<ipython-input-16-0109687a50a9>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  book_user_rating['unique_id_book'] = book_user_rating['ISBN'].map(d)


,ISBN,Book-Title,Book-Author,User-ID,Book-Rating,unique_id_book
0,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,110912,10,0
1,0440234743,The Testament,John Grisham,101209,7,1
2,0440234743,The Testament,John Grisham,133706,8,1
3,0440234743,The Testament,John Grisham,251122,9,1
4,0440234743,The Testament,John Grisham,54005,9,1
5,0440234743,The Testament,John Grisham,198046,8,1
6,0440234743,The Testament,John Grisham,108143,9,1
7,0440234743,The Testament,John Grisham,123430,5,1
8,0440234743,The Testament,John Grisham,44663,8,1
9,0440234743,The Testament,John Grisham,193529,7,1


In [17]:
users_books_pivot_matrix_df = book_user_rating.pivot(index='User-ID',
                                                        columns='unique_id_book',
                                                        values='Book-Rating').fillna(0)
users_books_pivot_matrix_df.head(10)

unique_id_book,0,1,2,3,4,5,6,7,8,9,...,15631,15632,15633,15634,15635,15636,15637,15638,15639,15640
User-ID,,,,,,,,,,,,,,,,,,,,,
14,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
51,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
56,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
109,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
165,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
242,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
243,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
users_books_pivot_matrix_df = users_books_pivot_matrix_df.values
users_books_pivot_matrix_df

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

Разложение исходной матрицы, это позволит нам выявить скрытые факторы, влияющие на оценки пользователями книг

In [19]:
NUMBER_OF_FACTORS_MF = 15

#Performs matrix factorization of the original user item matrix
U, sigma, Vt = svds(users_books_pivot_matrix_df, k = NUMBER_OF_FACTORS_MF)

In [20]:
sigma = np.diag(sigma)
sigma.shape

(15, 15)

Двойное матричное умножение для получения предположительных оценок от каждого пользователя к каждой книге

In [21]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt)
all_user_predicted_ratings

array([[ 1.67958901e-21,  1.63768329e-33,  9.64572211e-35, ...,
         1.69344811e-21,  9.42825463e-37, -1.34892895e-34],
       [ 1.13781723e-16, -2.77862554e-25, -9.36643875e-27, ...,
        -8.92634115e-18, -1.81769681e-28, -3.91378283e-27],
       [ 1.86171451e-20, -1.19969897e-33, -1.57400266e-34, ...,
        -1.58224668e-21, -3.70547186e-36,  6.28863139e-34],
       ...,
       [ 8.92995867e-21,  1.85105061e-33, -1.18826802e-34, ...,
        -1.07882394e-21, -1.88369777e-36,  2.01599993e-34],
       [ 2.87937495e-12, -1.81467246e-25,  2.65271777e-26, ...,
         1.65761012e-13,  6.89857218e-29, -5.28781008e-26],
       [ 1.19703996e-20,  2.52095669e-33,  9.17628371e-35, ...,
         2.93985583e-21,  3.21607469e-36, -9.05617193e-34]])

In [22]:
def top_cosine_similarity(data, book_id, top_n=10):
    index = book_id
    book_row = data[index, :]
    magnitude = np.sqrt(np.einsum('ij, ij -> i', data, data))
    similarity = np.dot(book_row, data.T) / (magnitude[index] * magnitude)
    sort_indexes = np.argsort(-similarity)
    return sort_indexes[:top_n]

def similar_books(book_user_rating, book_id, top_indexes):
    print('Recommendations for {0}: \n'.format(
    book_user_rating[book_user_rating.unique_id_book == book_id]['Book-Title'].values[0]))
    for id in top_indexes + 1:
        print(book_user_rating[book_user_rating.unique_id_book == id]['Book-Title'].values[0])

In [25]:
k = 50
book_id =1000
top_n = 3
sliced = Vt.T[:, :k] # representative data

similar_books(book_user_rating, 1000, top_cosine_similarity(sliced, book_id, top_n))

Recommendations for Curious George and the Pizza: 

Girl, Interrupted
Tango Charlie and Foxtrot Romeo: The Star Pit (Tor Double, No 4)
Marion DÃ¶nhoff: Ein widerstÃ¤ndiges Leben
